In [7]:
import os
from yt_dlp import YoutubeDL


def download_videos_one_by_one(channel_url, save_path="./", playlist_items='1-2', cookies_path="cookies.txt", max_duration=300):
    """
    从指定 YouTube 频道逐个获取符合条件的视频，下载视频及其缩略图。
    
    参数：
    - channel_url (str): 频道或播放列表的 URL。
    - save_path (str): 下载视频和缩略图保存的根路径。
    - playlist_items (str): 要提取的视频范围，例如 '1-2' 表示前两条。
    - cookies_path (str): 浏览器的 cookies 文件路径。
    - max_duration (int): 视频时长的最大值（秒），超出此时间的视频将被跳过。
    """
    try:

        # Step 1: 提取频道信息
        ydl_opts_info = {
            'quiet': False,  # 显示调试信息
            'extract_flat': True,  # 只提取视频信息，不下载视频
            'playlist_items': playlist_items,  # 选择指定范围的视频
        }
        with YoutubeDL(ydl_opts_info) as ydl:
            info_dict = ydl.extract_info(channel_url, download=False)
            videos = info_dict.get('entries', [])
            
            if not videos:
                print("No videos found.")
                return

        # Step 2: 逐个处理符合条件的视频
        for video in videos:
            video_url = video.get('url')
            video_title = video.get('title')
            video_duration = video.get('duration', 0)

            # 检查视频时长
            if video_duration > max_duration:
                print(f"Skipping video (too long): {video_title}")
                continue

            # 创建视频保存路径
            video_folder = save_path

            print(f"Downloading video: {video_title} ({video_url})")

            # 下载视频
            ydl_opts_video = {
                'outtmpl': os.path.join(video_folder, f"%(title)s.%(ext)s"),  # 视频保存路径
                'format': 'bestvideo+bestaudio/best',                       # 最佳视频+最佳音频
                'cookiesfrombrowser': ('chrome',),                         # 从浏览器加载 cookies
            }

            try:
                with YoutubeDL(ydl_opts_video) as ydl:
                    ydl.download([video_url])
                print(f"Download completed for: {video_title}")
            except Exception as e:
                print(f"Failed to download video: {video_title} ({e})")
                continue

            # 获取视频的缩略图 URL (从 thumbnails 列表中)
            thumbnails = video.get('thumbnails', [])
            if thumbnails:
                # 选择第一个有效的缩略图链接
                thumbnail_url = thumbnails[0]['url']
                try:
                    # 下载缩略图文件
                    thumbnail_path = os.path.join(video_folder, "thumbnail.jpg")
                    ydl_opts_thumbnail = {
                        'outtmpl': thumbnail_path,  # 缩略图保存路径
                        'format': 'best'  # 下载最佳画质
                    }
                    with YoutubeDL(ydl_opts_thumbnail) as ydl:
                        ydl.download([thumbnail_url])
                    print(f"Thumbnail downloaded for: {video_title}")
                except Exception as e:
                    print(f"Failed to download thumbnail for: {video_title} ({e})")
            else:
                print(f"No thumbnail found for: {video_title}")
            


    except Exception as e:
        print(f"An error occurred: {e}")


In [8]:
channel_url = "https://www.youtube.com/@NBA/videos"
save_path = "./downloads"
playlist_items = '1-1'  # 下载前 5 个视频及其封面
download_videos_one_by_one(channel_url, save_path, playlist_items)


[youtube:tab] Extracting URL: https://www.youtube.com/@NBA/videos
[youtube:tab] @NBA/videos: Downloading webpage
[download] Downloading playlist: NBA - Videos
[youtube:tab] Playlist NBA - Videos: Downloading 1 items
[download] Downloading item 1 of 1
[download] Finished downloading playlist: NBA - Videos
Extracting cookies from chrome
Extracted 3291 cookies from chrome
[youtube] Extracting URL: https://www.youtube.com/watch?v=edDPUe1MJdo
[youtube] edDPUe1MJdo: Downloading webpage
[youtube] edDPUe1MJdo: Downloading ios player API JSON
[youtube] edDPUe1MJdo: Downloading mweb player API JSON
[youtube] edDPUe1MJdo: Downloading m3u8 information
[info] Testing format 616
[info] edDPUe1MJdo: Downloading 1 format(s): 616+251
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 110
[download] Destination: ./downloads/GRIZZLIES at BULLS ｜ FULL GAME HIGHLIGHTS ｜ November 23, 2024.f616.mp4
[download] 100% of  360.89MiB in 00:02:57 at 2.04MiB/s                   
[download] Destinatio

[info] hqdefault: Downloading 1 format(s): 0
[download] Destination: ./downloads/thumbnail.jpg
[download] 100% of    7.67KiB in 00:00:00 at 13.40KiB/s  
Thumbnail downloaded for: GRIZZLIES at BULLS | FULL GAME HIGHLIGHTS | November 23, 2024
